In [36]:
import os
import numpy as np
import lsst.sims.maf.db as db

In [119]:
# connect to database and grab the needed columns from the summary table, without duplicates.
#runName = 'kraken_1040'
runName = 'aws1_1001'
dbDir = 'db'
opsdb = db.OpsimDatabase(os.path.join(dbDir, runName + '_sqlite.db'))

propid, proptags = opsdb.fetchPropInfo()
ddpropid = proptags['DD'][0]

# Query table.
cols = ['expMJD', 'night', 'fieldRA', 'fieldDec', 'rotSkyPos', 'filter',
        'visitExpTime', 'finSeeing', 'fiveSigmaDepth', 'solarElong', 
        'fieldID', 'propID']

obs = opsdb.fetchMetricData(cols, sqlconstraint='')

In [120]:
# Check that everything came back in time order..
idx = np.argsort(obs['expMJD'])
if (np.all(obs['expMJD'][idx] == obs['expMJD'])):
    print 'All in order!'
else:
    print "You need to check the order."

All in order!


In [121]:
# Join back-to-back visits.
nextObs = obs[1:]
currObs = obs[:-1]
match = np.where((nextObs['fieldID'] == currObs['fieldID']) & (nextObs['filter'] == currObs['filter']) & 
                 ((nextObs['expMJD'] - currObs['expMJD']) < 41./60./60./24) & 
                 (nextObs['expMJD'] - currObs['expMJD'] > 0) & (currObs['propID']!=ddpropid))[0]

In [122]:
print match

[      0       2       6 ..., 3750906 3750908 3750910]


In [125]:
newObs = np.copy(currObs)
newObs['visitExpTime'][match] = currObs['visitExpTime'][match] + nextObs['visitExpTime'][match]
newObs['finSeeing'][match] = np.max([currObs['finSeeing'][match], nextObs['finSeeing'][match]])
newObs['expMJD'][match] = (currObs['expMJD'][match] + nextObs['expMJD'][match]) / 2.0
newObs['fiveSigmaDepth'][match] = 1.25 *np.log10((10.**(0.8*currObs['fiveSigmaDepth'][match])) + 
                                                 10.**(0.8*nextObs['fiveSigmaDepth'][match]))

print newObs[0]
print newObs[match[0]]
print currObs[match[0]]

(1, 49353.034064499996, 0, 1.256637, 0.045991, 2.462996, 'z', 60.0, 2.226991, 22.077426731330824, 145.070664, 2766, 9)
(1, 49353.034064499996, 0, 1.256637, 0.045991, 2.462996, 'z', 60.0, 2.226991, 22.077426731330824, 145.070664, 2766, 9)
(1, 49353.033856, 0, 1.256637, 0.045991, 2.462996, 'z', 30.0, 1.075637, 21.700632, 145.070664, 2766, 9)


In [126]:
formatstring = ['%d', '%g', '%d', '%g', '%g', '%f', '%s', '%.1f', '%f', '%f', '%f', '%d', '%d']
header = ','.join(cols)
header = '#line,' + header
print header

#line,expMJD,night,fieldRA,fieldDec,rotSkyPos,filter,visitExpTime,finSeeing,fiveSigmaDepth,solarElong,fieldID,propID


In [127]:
np.savetxt(runName+'_summary.csv', newObs, fmt=formatstring, delimiter=',', header=header, comments='')

In [128]:
!ls $runName"_summary.csv"
!head $runName'_summary.csv'

aws1_1001_summary.csv
#line,expMJD,night,fieldRA,fieldDec,rotSkyPos,filter,visitExpTime,finSeeing,fiveSigmaDepth,solarElong,fieldID,propID
1,49353,0,1.25664,0.045991,2.462996,z,60.0,2.226991,22.077427,145.070664,2766,9
2,49353,0,1.25664,0.045991,2.462996,z,30.0,1.074755,21.701646,145.070319,2766,11
3,49353,0,1.20101,0.046915,2.514462,z,60.0,2.226991,22.100065,142.555308,2776,9
4,49353,0,1.20101,0.046915,2.514462,z,30.0,1.055212,21.724271,142.554949,2776,11
5,49353,0,1.17385,0.004883,2.512492,z,30.0,1.030315,21.753590,139.874600,2664,9
6,49353,0,1.22901,0.003271,2.457322,z,30.0,1.044518,21.736795,142.301424,2654,9
7,49353,0,1.31227,0.046915,2.415675,z,60.0,2.226991,22.068543,147.556582,2772,9
8,49353,0,1.31227,0.046915,2.415675,z,30.0,1.073145,21.701176,147.556253,2772,11
9,49353,0,1.33943,0.004882,2.365949,z,30.0,1.064461,21.711234,147.041715,2662,9
